<b>General implementation of a Quantum PCA for any size covariance matrices.</b>

In [8]:
import numpy as np
import scipy
import copy
from math import *
from qiskit import(
  QuantumCircuit,
  execute,
  BasicAer,
  QuantumRegister,
  ClassicalRegister)
from qiskit.visualization import plot_histogram
from qiskit.quantum_info.synthesis import euler_angles_1q 
from qiskit.quantum_info.operators import Operator
from qiskit.aqua.utils.controlled_circuit import get_controlled_circuit

Utility functions

In [2]:
def encode_eigv2(eigv):
  print(eigv)
  return (2*acos(eigv[0]), 0, 0)

def encode_covmat(covmat):
  covmat=covmat/np.trace(covmat)
  u=np.round(scipy.linalg.expm(2*np.pi*1j*np.array(covmat)),4)
  return euler_angles_1q(u)

#FIXME: controllare che funzioni per più qbit per psi
def eigval_from(count, psibits):
    occurrence = {}
    for i in count.items():
        substring = i[0][psibits:]
        if substring not in occurrence.keys():
            occurrence[substring] = i[1]
        else: 
            occurrence[substring] += i[1]
    maxcount = max(occurrence.values())
    binary_res = None
    acc=0
    for i in occurrence.items():
        if i[1]==maxcount:
            binary_res=i[0]
    for i in range(len(binary_res)):
        acc+=2**(-i-1)*int(binary_res[i])
    return acc

Parametric implementation of the algorithm

In [156]:
#Check if valid matrix, add rows and cols in order to make it of 2**i length, return n° of bits required to encode it
def preprocess_mat(matrix):
    if matrix is None or len(matrix)!=len(matrix[0]) or len(matrix)==0:
        raise ValueError()
    while len(matrix) & (len(matrix) - 1) != 0:
        b = np.zeros((len(matrix)+1,len(matrix)+1))
        b[:-1,:-1] = matrix
        matrix=b
    PSIBITS = ceil(log2(len(matrix)))
    matrix = scipy.linalg.expm(2*np.pi*1j*np.array(matrix))
    return matrix, PSIBITS

def generate_cu3(unitary):
    op = np.eye(2*len(unitary),dtype="complex")
    for i in range(len(op)//2, len(op)):
        for j in range(len(op)//2, len(op)):
            op[i,j] = unitary[i-len(op)//2,j-len(op)//2]
    return Operator(op)
    
#Creates the circuit needed to caluclate eigenvalues/eigenvectors
def generate_circuit(initial, covmat, NBITS, PSIBITS):
    circuit = QuantumCircuit(NBITS+PSIBITS, NBITS+PSIBITS)
    for i in range(NBITS):
        circuit.h(i)
    circuit.initialize(initial, [i for i in range(NBITS,NBITS+PSIBITS)])
    #Phase kickback:
    cu3 = generate_cu3(covmat)
    qubits = circuit.qubits
    for i in range(NBITS):
        for j in range(2**i):
            q = [qubits[k] for k in range(NBITS, NBITS+PSIBITS)] + [qubits[NBITS-i-1]]
            circuit.unitary(cu3,q)
    circuit.barrier()        
    
    result = execute(circuit, BasicAer.get_backend('statevector_simulator')).result()
    #print(np.round(result.get_statevector(circuit),4))
    #inverse QFT:
    for i in range(NBITS):
        circuit.h(i)
        for j in range(1,NBITS-i):
            circuit.cu1(-pi/(2**(j)),i,j+i)
        circuit.barrier()
    circuit.measure([i for i in range(NBITS+PSIBITS)],[i for i in range(NBITS+PSIBITS)])
    return circuit

#From the results of the measure, estimate eigenvector for next measure
#FIXME hardcoded
def estimate_eigv(counts,PSIBITS):
    keys = list(counts.keys())
    if PSIBITS==1:
        zero = sum([counts[k] for k in keys if k[0]=='0'])
        ones = sum([counts[k] for k in keys if k[0]=='1'])
        pz = sqrt(zero/(zero+ones))
        po = sqrt(ones/(zero+ones))
        return [pz, po]
    elif PSIBITS==2:
        zz = sqrt(sum([counts[i] for i in counts.keys() if i[0:2]=='00'])/sum([counts[k] for k in keys]))
        zo = sqrt(sum([counts[i] for i in counts.keys() if i[0:2]=='01'])/sum([counts[k] for k in keys]))
        oz = sqrt(sum([counts[i] for i in counts.keys() if i[0:2]=='10'])/sum([counts[k] for k in keys]))
        oo = sqrt(sum([counts[i] for i in counts.keys() if i[0:2]=='11'])/sum([counts[k] for k in keys]))
        return [zz,zo,oz,oo]/ np.linalg.norm([zz,zo,oz,oo])
    else:
        print("This function works only for 2x2 now")
        raise ValueError()

def generatefirst(psibits, realdim):
    last_approx=[0]*2**psibits
    for i in range(realdim):
        last_approx[i] = 1
    return last_approx / np.linalg.norm(last_approx)
    

def qpca(covmat, NBITS, initialeig = None, iterations=1, simulator=BasicAer.get_backend('qasm_simulator'), req_shots=8192):
    iteration=0
    REALDIM=len(covmat)
    covmat, PSIBITS = preprocess_mat(covmat)
    last_approx = generatefirst(PSIBITS, REALDIM)
    print("last_approx", last_approx)
    while(iteration<iterations):
        circuit = generate_circuit(last_approx, covmat, NBITS, PSIBITS)
        job = execute(circuit, simulator, shots=req_shots)
        result = job.result().data(circuit)
        counts = job.result().get_counts()
        last_approx = estimate_eigv(counts, PSIBITS)
        print("last_approx",last_approx)
        iteration+=1
    return {"results": counts, "eigvec": last_approx, "eigval": eigval_from(counts,PSIBITS), "lastcirc": circuit}

Generation of the covariance matrix and normalization

In [165]:
A = scipy.random.rand(2,2)
covmat = np.dot(A,A.transpose())
covmat=covmat/np.trace(covmat)
print(covmat)

[[0.97447078 0.15129853]
 [0.15129853 0.02552922]]


In [166]:
eigval, eigvec = np.linalg.eig(covmat)
print("Eigenvalues: ")
print(eigval)
print("Eigenvectors: ")
print(eigvec)

Eigenvalues: 
[0.9980098 0.0019902]
Eigenvectors: 
[[ 0.9881128  -0.15373059]
 [ 0.15373059  0.9881128 ]]


In [168]:
res=qpca(covmat,10,iterations=7)
print(res["eigval"])
# PLOT THE RESULTS HISTOGRAM
#plot_histogram(res["results"])
# SHOW THE CIRCUIT AT LAST ITERATION
#res["lastcirc"].draw(fold=200)

last_approx [0.70710678 0.70710678]
last_approx [0.7964153085859161, 0.6047500775113633]
last_approx [0.8755578578826188, 0.4831132760543846]
last_approx [0.9271564811966748, 0.3746743377588062]
last_approx [0.9606516343087124, 0.27775607554111215]
last_approx [0.9813321024569104, 0.19232083789204954]
last_approx [0.9861714758030167, 0.16572815184059708]
last_approx [0.9866664797121163, 0.16275520824999734]
0.998046875
